#### TODO
1. Schedule Camp Status Check    
    1.1 自动检查每日应该运行的Camp    
    1.2 记录检查结果，并将失败Camp反馈给对应负责人

In [1]:
import pandas as pd
import cx_Oracle
import datetime

def get_res(communication_cd, url_check, launch_time):
    db = cx_Oracle.connect('HCCN_LEOZHANGZS','yATbUPR64L6A','SASXRCN.HOMECREDIT.CN') #连接数据库
    con = db.cursor() # 游标操作
    oo = f"select ge.campaign_cd,\
    ge.campaign_nm,ge.communication_cd,\
    ge.communication_nm,ge.communication_occurrence_no,\
    ge.channel_cd,ge.subject_type_nm,ge.processed_dttm \
    from app_campaign_cdm.ci_cell_package ge \
    where ge.communication_cd = '{communication_cd}' \
    and ge.processed_dttm > TO_DATE('{launch_time}', 'yyyy-mm-dd hh24:mi:ss')"
    con.execute(oo)
    rows = con.fetchall()
    object_type = rows[0][6]
    occurrence_no = rows[0][4]
    launch_time = rows[0][7].strftime("%Y-%m-%d %X")
    channel_cd = rows[0][5]
    owner_num = owner_cnt(con, channel_cd, communication_cd, occurrence_no, launch_time)
    if rows[0][5] == "SMT":
        res_smt(con, communication_cd, object_type, occurrence_no, launch_time, url_check)

def res_smt(con, communication_cd, object_type, occurrence_no, launch_time, url_check):
    contact_num = contact_cnt(con, communication_cd, object_type, launch_time)
    inbox_num = inbox_cnt(con, communication_cd, object_type, launch_time)
    if url_check == 'a':
        url_click_num = url_click(con, communication_cd, object_type, launch_time)

    tem_type_sql = f"select case when tem.is_popup > 0 then 'POP_UP'  when  tem.is_notify > 0 then 'PUSH' end as Notify_type \
    from app_campaign_cdm.ci_cell_package ge \
    left join OWNER_OUT.OT_SMT_MESSAGE_DATA TA \
    on ta.campaign_track_id = ge.response_tracking_cd \
    left join dm_campaign.v_smt_template tem \
    on ta.template_id = tem.id \
    where ge.communication_cd = '{communication_cd}' and ge.processed_dttm > TO_DATE('{launch_time}', 'yyyy-mm-dd hh24:mi:ss')"
    
    if tem_type == "POP_UP":
        pop_num = popup_cnt(con, communication_cd, object_type, occurrence_no, launch_time)
    else:
        push_num = push_cnt(con, communication_cd, object_type, occurrence_no, launch_time)
    

def owner_cnt(con, channel_cd, communication_cd, occurrence_no, launch_time):
    if channel_cd == "SMT":
        oo_tb = ""
    sql = f"SELECT COUNT(DISTINCT TA.ID_ENTITY_EMPLOYEE) AS OWNER_OUT\
    FROM OWNER_OUT.{oo_tb} TA\
    LEFT JOIN APP_CAMPAIGN_CDM.CI_CELL_PACKAGE GE\
    ON GE.RESPONSE_TRACKING_CD = TA.CAMPAIGN_TRACK_ID\
    WHERE GE.COMMUNICATION_CD IN ('{communication_cd}')\
    and GE.Communication_Occurrence_No  = {occurrence_no}\
    AND TA.DATE_EFFECTIVE >= TO_DATE('{launch_time}','yyyy-mm-dd hh24:mi:ss')"
    con.execute(sql)
    rows = con.fetchall()
    print(rows)

def contact_cnt(con, communication_cd, object_type, launch_time):
    pass

def inbox_cnt(con, communication_cd, object_type, occurrence_no ,launch_time):
    a = lambda x: "TIP" if x == "Tipper" else "EMP"
    sql = f"SELECT COUNT(DIstinct hmp.id_xrm_employee) as INBOX_RATE\
    FROM APP_CAMPAIGN_CDM.CI_RESPONSE_HISTORY_{a(object_type)} HMP\
    JOIN APP_CAMPAIGN_CDM.CI_CELL_PACKAGE GE\
    ON GE.CELL_PACKAGE_SK = HMP.CELL_PACKAGE_SK\
    WHERE GE.COMMUNICATION_CD = '{communication_cd}'\
    AND HMP.RESPONSE_SK IN ('1029', '1030')\
	AND ge.communication_occurrence_no = {occurrence_no}\
	AND HMP.RESPONSE_DTTM >= TO_DATE('{launch_time}', 'yyyy-mm-dd hh24:mi:ss')"
    con.execute(sql)
    rows = con.fetchall()
    print(rows)

def popup_cnt(con, communication_cd, object_type, occurrence_no ,launch_time):
    a = lambda x: "TIP" if x == "Tipper" else "EMP"
    sql = f"SELECT COUNT(DIstinct hmp.id_xrm_employee) as INBOX_RATE\
    FROM APP_CAMPAIGN_CDM.CI_RESPONSE_HISTORY_{a(object_type)} HMP\
    JOIN APP_CAMPAIGN_CDM.CI_CELL_PACKAGE GE\
    ON GE.CELL_PACKAGE_SK = HMP.CELL_PACKAGE_SK\
    WHERE GE.COMMUNICATION_CD = '{communication_cd}'\
    AND HMP.RESPONSE_SK = '1028'\
	AND ge.communication_occurrence_no = {occurrence_no}\
	AND HMP.RESPONSE_DTTM >= TO_DATE('{launch_time}', 'yyyy-mm-dd hh24:mi:ss')"
    con.execute(sql)
    rows = con.fetchall()
    print(rows)

def push_cnt(con, communication_cd, object_type, occurrence_no ,launch_time):
    a = lambda x: "TIP" if x == "Tipper" else "EMP"
    sql = f"SELECT COUNT(DIstinct hmp.id_xrm_employee) as INBOX_RATE\
    FROM APP_CAMPAIGN_CDM.CI_RESPONSE_HISTORY_{a(object_type)} HMP\
    JOIN APP_CAMPAIGN_CDM.CI_CELL_PACKAGE GE\
    ON GE.CELL_PACKAGE_SK = HMP.CELL_PACKAGE_SK\
    WHERE GE.COMMUNICATION_CD = '{communication_cd}'\
    AND HMP.RESPONSE_SK = '1031'\
	AND ge.communication_occurrence_no = {occurrence_no}\
	AND HMP.RESPONSE_DTTM >= TO_DATE('{launch_time}', 'yyyy-mm-dd hh24:mi:ss')"
    con.execute(sql)
    rows = con.fetchall()
    print(rows)

def url_click(con, communication_cd, object_type, launch_time):
    pass

In [2]:
act_list = pd.read_csv(r'C:\Users\leo.zhangzs\AppData\Roaming\Code\User\VScode\py3\schedule_monitor\db\Camp_job.csv')
# act_list['LAUNCH_TIME'] = pd.to_datetime(act_list['LAUNCH_TIME'], format = '%Y/%m/%d %H:%M')
inv_time = (datetime.datetime.now()+datetime.timedelta(days=-20)).strftime('%Y-%m-%d %X')
print(inv_time)

2019-10-08 10:08:10


In [5]:
act_list[["CAMPAIGN_CD","LAUNCH_TIME"]]

,CAMPAIGN_CD,LAUNCH_TIME
0,XNA,2019-09-24 16:00:00
1,XNA,2019-09-24 16:00:00
2,XNA,2019-09-24 00:00:00
3,XNA,2019-09-24 00:00:00
4,XNA,2019-10-15 16:00:00
5,XNA,2019-10-15 16:00:00
6,XNA,2019-10-17 10:48:00
7,XNA,2019-10-18 15:00:00
8,XNA,2019-10-18 15:00:00
9,XNA,2019-10-22 16:00:00


In [68]:
df = act_list[(act_list['CHECK_STATUS'] == 'n') & (act_list['LAUNCH_TIME'] > inv_time)]
df

,COMMUNICATION_CD,LAUNCH_TIME,TITLE_NAME,URL_CHICK,CHECK_STATUS,Sender
2,COMM4076,2019/9/24 0:00,Tipper 2nd ACL Lucky Draw have contract group,a,n,Leo.ZhangZS@homecredit.cn
3,COMM4121,2019/9/24 0:00,Tipper 2nd ACL Lucky Draw no contract group,a,n,Leo.ZhangZS@homecredit.cn


In [70]:
for i in df.index:
    communication_cd = df[df.index==i]['COMMUNICATION_CD'].to_string(index=None).strip()
    url_chick = df[df.index==i]['URL_CHICK'].to_string(index=None).strip()
    launch_time = df[df.index==i]['LAUNCH_TIME'].to_string(index=None).strip()
#     get_res(communication_cd, url_chick, launch_time)
#     print(communication_cd, url_chick, launch_time)
    act_list.loc[i,'CHECK_STATUS'] = 'y'
act_list

,COMMUNICATION_CD,LAUNCH_TIME,TITLE_NAME,URL_CHICK,CHECK_STATUS,Sender
0,COMM4088,2019/9/24 16:00,Tipper 2nd ACL Lucky Draw have contract group,n,y,Leo.ZhangZS@homecredit.cn
1,COMM4124,2019/9/24 16:00,Tipper 2nd ACL Lucky Draw no contract group,n,y,Leo.ZhangZS@homecredit.cn
2,COMM4076,2019/9/24 0:00,Tipper 2nd ACL Lucky Draw have contract group,a,y,Leo.ZhangZS@homecredit.cn
3,COMM4121,2019/9/24 0:00,Tipper 2nd ACL Lucky Draw no contract group,a,y,Leo.ZhangZS@homecredit.cn


In [82]:
import os, datetime

if os.path.exists(r'C:\Program Files (x86)\WinSCP\WinSCP.com'):
    winscp = r'C:\Program Files (x86)\WinSCP\WinSCP.com'
elif os.path.exists(r'D:\Program Files (x86)\WinSCP\WinSCP.com'):
    winscp = r'D:\Program Files (x86)\WinSCP\WinSCP.com'
elif os.path.exists(r'D:\Program Files (x86)\WinSCP\WinSCP.com'):
    winscp = r'D:\Program Files (x86)\WinSCP\WinSCP.com'
    
winscp

'D:\\Program Files (x86)\\WinSCP\\WinSCP.com'

#### D-3 Camp Reports
##### TODO LIST
1. Get List(after SDM updated)
2. Get campaign results
3. package results
4. create a report and attachments
5. send e-mail

In [1]:
import pandas as pd
import cx_Oracle
import datetime

db_user = 'HCCN_LEOZHANGZS'
db_password = 'yATbUPR64L6A'
db_server = 'SASXRCN.HOMECREDIT.CN'

def camp_list(con, Channel_cd):
    sql =f"""select cie.campaign_cd, cie.communication_cd,cie.wave_number, ge.campaign_nm,  ge.cell_package_sk, cie.primary_export_count, cie.exported_subjects_count, cie.execution_timestamp, cie.channel_cd from app_campaign_cdm.ci_cell_package ge 
        left join app_campaign_cdm.cie_int_execution cie
        on cie.communication_cd = ge.communication_cd and cie.wave_number = ge.communication_occurrence_no
        where trunc(cie.execution_timestamp) between sysdate-4 and sysdate-1
        and cie.status = 'CONSUMED'
        and cie.channel_cd = '{Channel_cd}'
        and not regexp_like(ge.campaign_nm, 'test', 'i')
        order by cie.execution_timestamp DESC"""
    
    con.execute(sql)
    rows = con.fetchall()
    return rows

def camp_inbox(con,cell_package_sk):
    sql = f"""with tip as (select distinct hmp.id_xrm_employee,HMP.CELL_PACKAGE_SK
          FROM APP_CAMPAIGN_CDM.CI_RESPONSE_HISTORY_TIP HMP
         WHERE HMP.CELL_PACKAGE_SK IN ({cell_package_sk})
           AND HMP.RESPONSE_SK IN ('1029', '1030')),emp as (select distinct hmp.id_xrm_employee,HMP.CELL_PACKAGE_SK
          FROM APP_CAMPAIGN_CDM.CI_RESPONSE_HISTORY_EMP HMP
         WHERE HMP.CELL_PACKAGE_SK IN ({cell_package_sk})
           AND HMP.RESPONSE_SK IN ('1029', '1030'))
        select a.CELL_PACKAGE_SK,COUNT(1) Inbox_Read from tip a group by a.CELL_PACKAGE_SK
        union all
        select b.CELL_PACKAGE_SK,COUNT(1) Inbox_Read from emp b group by b.CELL_PACKAGE_SK
        order by CELL_PACKAGE_SK DESC"""
    
    con.execute(sql)
    rows = con.fetchall()
    indexs= [i[0] for i in rows]
    datas = [i[1] for i in rows]
    df = pd.DataFrame(datas, index = indexs, columns=['INBOX_READ'])
    df['INBOX_READ'] = pd.to_numeric(df['INBOX_READ'], errors="coerce", downcast="integer")
    return df

def camp_popup(con,cell_package_sk):
    sql = f"""with tip as (select distinct hmp.id_xrm_employee,HMP.CELL_PACKAGE_SK
          FROM APP_CAMPAIGN_CDM.CI_RESPONSE_HISTORY_TIP HMP
         WHERE HMP.CELL_PACKAGE_SK IN ({cell_package_sk})
           AND HMP.RESPONSE_SK = '1028'),emp as (select distinct hmp.id_xrm_employee,HMP.CELL_PACKAGE_SK
          FROM APP_CAMPAIGN_CDM.CI_RESPONSE_HISTORY_EMP HMP
         WHERE HMP.CELL_PACKAGE_SK IN ({cell_package_sk})
           AND HMP.RESPONSE_SK = '1028')
        select a.CELL_PACKAGE_SK,COUNT(1) Inbox_Read from tip a group by a.CELL_PACKAGE_SK
        union all
        select b.CELL_PACKAGE_SK,COUNT(1) Inbox_Read from emp b group by b.CELL_PACKAGE_SK
        order by CELL_PACKAGE_SK DESC"""
    
    con.execute(sql)
    rows = con.fetchall()
    indexs= [i[0] for i in rows]
    datas = [i[1] for i in rows]
    df = pd.DataFrame(datas, index = indexs, columns=['POPUP_READ'])
    df['POPUP_READ'] = pd.to_numeric(df['POPUP_READ'], errors="coerce", downcast="integer")
    return df

def camp_push(con,cell_package_sk):
    sql = f"""with tip as (select distinct hmp.id_xrm_employee,HMP.CELL_PACKAGE_SK
          FROM APP_CAMPAIGN_CDM.CI_RESPONSE_HISTORY_TIP HMP
         WHERE HMP.CELL_PACKAGE_SK IN ({cell_package_sk})
           AND HMP.RESPONSE_SK = '1031'),emp as (select distinct hmp.id_xrm_employee,HMP.CELL_PACKAGE_SK
          FROM APP_CAMPAIGN_CDM.CI_RESPONSE_HISTORY_EMP HMP
         WHERE HMP.CELL_PACKAGE_SK IN ({cell_package_sk})
           AND HMP.RESPONSE_SK = '1031')
        select a.CELL_PACKAGE_SK,COUNT(1) Inbox_Read from tip a group by a.CELL_PACKAGE_SK
        union all
        select b.CELL_PACKAGE_SK,COUNT(1) Inbox_Read from emp b group by b.CELL_PACKAGE_SK
        order by CELL_PACKAGE_SK DESC"""
    
    con.execute(sql)
    rows = con.fetchall()
    indexs= [i[0] for i in rows]
    datas = [i[1] for i in rows]
    df = pd.DataFrame(datas, index = indexs, columns=['PUSH_READ'])
    df['PUSH_READ'] = pd.to_numeric(df['PUSH_READ'], errors="coerce", downcast="integer")
    return df

def camp_url(con,cell_package_sk):
    sql = f"""with tip as (select distinct hmp.id_xrm_employee,HMP.CELL_PACKAGE_SK
          FROM APP_CAMPAIGN_CDM.CI_RESPONSE_HISTORY_TIP HMP
         WHERE HMP.CELL_PACKAGE_SK IN ({cell_package_sk})
           AND HMP.External_Response_Info_Id2 is not null),emp as (select distinct hmp.id_xrm_employee,HMP.CELL_PACKAGE_SK
          FROM APP_CAMPAIGN_CDM.CI_RESPONSE_HISTORY_EMP HMP
         WHERE HMP.CELL_PACKAGE_SK IN ({cell_package_sk})
           AND HMP.External_Response_Info_Id2 is not null)
        select a.CELL_PACKAGE_SK,COUNT(1) Inbox_Read from tip a group by a.CELL_PACKAGE_SK
        union all
        select b.CELL_PACKAGE_SK,COUNT(1) Inbox_Read from emp b group by b.CELL_PACKAGE_SK
        order by CELL_PACKAGE_SK DESC"""
    
    con.execute(sql)
    rows = con.fetchall()
    indexs= [i[0] for i in rows]
    datas = [i[1] for i in rows]
    df = pd.DataFrame(datas, index = indexs, columns=['URL_CLICK'])
    df['URL_CLICK'] = pd.to_numeric(df['URL_CLICK'], errors="coerce", downcast="integer")
    return df

In [2]:
def fun(data):
    df_sche = pd.read_csv(r'C:\Users\leo.zhangzs\AppData\Roaming\Code\User\VScode\py3\schedule_monitor\db\Schedule_job.csv', encoding='utf-8')
    df_cd = list(df_sche['COMMUNICATION_CD'])
    df_res = []
    for i in data:
        if i in df_cd:
            df_res.append('SCHEDULE')
        else:
            df_res.append('ONE TIME')
    return df_res

db = cx_Oracle.connect(db_user, db_password, db_server) #连接数据库
con = db.cursor() # 游标操作
res = camp_list(con, "SMT")
indexs= [i[4] for i in res]
df = pd.DataFrame(data=res, index=indexs, columns= ['CAMPAIGN_CD', 'COMMUNICATION_CD', 'WAVE_NUM', 'CAMPAIGN_NM', 'CELL_PACKAGE_SK', 'PRIMARY_EXPORT_COUNT', 'EXPORTED_SUBJECTS_COUNT', 'EXECUTION_TIMESTAMP', 'CHANNEL_CD'])

df_type = lambda x : fun(x)
# df['JOB_TYPE'] = df.apply(lambda x: 'SCHEDULE' if x.COMMUNICATION_CD in list(df_sche['COMMUNICATION_CD']) else "ONE TIME", axis = 1, raw = True)
df['JOB_TYPE'] = df_type(list(df['COMMUNICATION_CD']))
pd.DataFrame().empty
import os
os.path.basename

<function ntpath.basename(p)>

In [3]:
db = cx_Oracle.connect(db_user, db_password, db_server) #连接数据库
con = db.cursor() # 游标操作
pstr = lambda x: "'" + "','".join([str(i) for i in x]) + "'" 

df_inbox = camp_inbox(con, pstr(list(df['CELL_PACKAGE_SK'])))
# df_inbox = df_inbox.fillna(0)
df_popup = camp_popup(con, pstr(list(df['CELL_PACKAGE_SK'])))
df_push = camp_push(con, pstr(list(df['CELL_PACKAGE_SK'])))
df_url = camp_url(con, pstr(list(df['CELL_PACKAGE_SK'])))
db.close()

df_res = pd.concat([df, df_inbox], axis=1, join_axes=[df.index])
df_res = df_res.fillna(0)
df_res = pd.concat([df_res, df_popup, df_push, df_url], axis=1, join_axes=[df.index])
# df_res['INBOX_READ'] = pd.to_numeric(df_res['INBOX_READ'], errors="coerce", downcast="integer")
# .loc[df['a'].notnull(), 'a'] = df.loc[df['a'].notnull(), 'a'].astype(int)
# df_res.loc[df_res['POPUP_READ'].notnull(), 'POPUP_READ'] = df_res.loc[df_res['POPUP_READ'].notnull(), 'POPUP_READ'].astype(int)
df_res

,CAMPAIGN_CD,COMMUNICATION_CD,WAVE_NUM,CAMPAIGN_NM,CELL_PACKAGE_SK,PRIMARY_EXPORT_COUNT,EXPORTED_SUBJECTS_COUNT,EXECUTION_TIMESTAMP,CHANNEL_CD,JOB_TYPE,INBOX_READ,POPUP_READ,PUSH_READ,URL_CLICK
18117,CAMP1134,COMM4149,51,SMT_AVS_Welcome_New_Tipper_SCR-708,18117,179,179,2019-11-14 15:45:43,SMT,SCHEDULE,6.0,17,NaN,NaN
18116,CAMP347,COMM793,230,SMT_WELCOME_New_Tipper,18116,4008,4008,2019-11-14 15:31:02,SMT,SCHEDULE,92.0,289,NaN,NaN
18112,CAMP1051,COMM3765,56,SMT_ALDI2.0_Welcome_New_Tipper_SCR-649,18112,6,6,2019-11-14 14:30:43,SMT,SCHEDULE,0.0,1,NaN,NaN
18108,CAMP521,COMM1536,169,SMT_ACL_Introduction_To_New_Tipper_SCR-316,18108,1395,1395,2019-11-14 13:11:12,SMT,SCHEDULE,17.0,51,NaN,NaN
18107,CAMP521,COMM5164,3,SMT_ACL_Introduction_To_New_Tipper_SCR-316,18107,1019,1019,2019-11-14 13:10:47,SMT,ONE TIME,14.0,46,NaN,NaN
18072,CAMP1134,COMM4149,50,SMT_AVS_Welcome_New_Tipper_SCR-708,18072,167,167,2019-11-13 15:45:42,SMT,SCHEDULE,7.0,29,NaN,NaN
18071,CAMP347,COMM793,229,SMT_WELCOME_New_Tipper,18071,2739,2739,2019-11-13 15:30:54,SMT,SCHEDULE,85.0,358,NaN,NaN
18056,CAMP1051,COMM3765,55,SMT_ALDI2.0_Welcome_New_Tipper_SCR-649,18056,11,11,2019-11-13 14:30:42,SMT,SCHEDULE,0.0,1,NaN,NaN
18054,CAMP1304,COMM5214,1,SMT_ETC_Promotion_Nationalwide_Tipper_SCR-845,18054,559471,559471,2019-11-13 14:27:42,SMT,ONE TIME,4816.0,29916,NaN,NaN
18053,CAMP521,COMM1536,168,SMT_ACL_Introduction_To_New_Tipper_SCR-316,18053,1589,1589,2019-11-13 13:11:12,SMT,SCHEDULE,15.0,99,NaN,NaN


In [232]:
df_res['INBOX_RATE'] = df_res.apply(lambda x : '%.2f%%' % (x['INBOX_READ'] / x['EXPORTED_SUBJECTS_COUNT']*100) if x['INBOX_READ'] > 0 else x['INBOX_READ'], axis=1)
df_res['POPUP_RATE'] = df_res.apply(lambda x : '%.2f%%' % (x['POPUP_READ'] / x['EXPORTED_SUBJECTS_COUNT']*100) if x['POPUP_READ'] > 0 else x['POPUP_READ'], axis=1)
df_res['PUSH_RATE'] = df_res.apply(lambda x : '%.2f%%' % (x['PUSH_READ'] / x['EXPORTED_SUBJECTS_COUNT']*100) if x['PUSH_READ'] > 0 else x['PUSH_READ'], axis=1)
df_res['DELIVERY_RATE'] = df_res.apply(lambda x : '%.2f%%' % (x['PRIMARY_EXPORT_COUNT'] / x['EXPORTED_SUBJECTS_COUNT']*100) if x['PRIMARY_EXPORT_COUNT'] > 0 else x['PRIMARY_EXPORT_COUNT'], axis=1)
df_res['URL_CLICK_RATE'] = df_res.apply(lambda x : '%.2f%%' % (x['URL_CLICK'] / x['EXPORTED_SUBJECTS_COUNT']*100) if x['URL_CLICK'] > 0 else x['URL_CLICK'], axis=1)

In [239]:
import numpy as np
from math import isnan

fdir=r'C:\Users\leo.zhangzs\AppData\Roaming\Code\User\VScode\py3\schedule_monitor\db\D3_Report\test.html'
css_dir = r'C:\Users\leo.zhangzs\AppData\Roaming\Code\User\VScode\py3\schedule_monitor\conf\CSS\css_style.txt'
with open(css_dir, 'r') as f_css:
    myClass = f_css.read()
#     print(myClass)
f_css.close()
html_h2 = "<h2>SMT 2019/10/27 - 2019/10/30 Analysis</h2>"
with open(fdir, 'w', encoding='utf-8') as f:
    f.write(html_h2)
    f.write(df_res.to_html(columns=['EXECUTION_TIMESTAMP','CAMPAIGN_NM','EXPORTED_SUBJECTS_COUNT','DELIVERY_RATE','INBOX_READ','INBOX_RATE','POPUP_READ','POPUP_RATE','PUSH_READ','PUSH_RATE','URL_CLICK','URL_CLICK_RATE'],
                           bold_rows=False, na_rep='', float_format = lambda x: 1 if x != np.nan else x, index=0, classes='fl-table'))
    f.write(myClass)
f.close()

TypeError: object of type 'int' has no len()